In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from feature_engine.imputation import MeanMedianImputer

## Imputing  Missing Values : 

#### 1. CCA : Complete Case Analysis : 


In [ ]:
df.dropna()
df.dropna(subset=[column_names])
df.dropna(how="all")

#### 2. SimpleImputers : 

In [ ]:
from sklearn.impute import SimpleImputer 
imputer1 = SimpleImputer(strategy='mean')
imputer2 = SimpleImputer(strategy='median')
imputer1.fit(x)
imputer1.transform(x)


#### 3. MeanMedianImputer : 

In [ ]:
from feature_engine.imputation import MeanMedianImputer
median_imputer = MeanMedianImputer(imputation_method="median")
mean_imputer = MeanMedianImputer(imputation_method="mean")
median_imputer.fit(x)
median_imputer.transform(x)
median_imputer.imputer_dict_

#### 3.Column Transformer : 

In [ ]:
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer 

pipeline = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='mean'))
])

columnTransform = ColumnTransformer(transformers=  [
    
    ("simple_imputer",pipeline,numerical_columns)
])

columnTransform.fit_transform(x)


#### 4. Most Frequent Category or Mode : 
- Used for numeric as well as categorical features

In [ ]:
imputer1 = SimpleImputer(strategy="most_frequent")

#### 5. Imputation using Arbitrary Values : 
- used for numeric features 
- replacing missing values using any random value

In [ ]:
from sklearn.impute import SimpleImputer 
from feature_engine.missing_data_imputers import ArbitraryNumberImputer
imputer = SimpleImputer(strategy='constant', fill_value=99)
imputer.fit_transform(x)

imputer = ArbitraryNumberImputer(arbitrary_number=99, variables=[column-names])
imputer.fit_transform(x)

#### 6. Imputation using bespoke(custom-made) category : 
- Imputing missing values with "Missing" string

In [ ]:
from sklearn.impute import SimpleImputer 
from feature_engine.missing_data_imputers import CategoricalVariableImputer

imputer = SimpleImputer(strategy='constant', fill_value="Missing")
imputer.fit_transform(x)

imputer = CategoricalVariableImputer(variables=[column-names])
imputer.fit_transform(x)

#### 7.End of Tail imputation : 
- in this technique we impute missing values using IQR rule or standard formula. 
- value = df.quantile(0.75) + 1.5*IQR  --> for rightward values in dataset 
- value = df.quantile(0.25) - 1.5*IQR  --> for leftward values in dataset 
- value = df.mean() + 3*df.std() 
- or simply with , value = df.max()*3 
- Do not preserve original distribution

In [ ]:
from feature_engine.missing_data_imputers import EndTailImputer
## for rightward skewed dataset --> df.quantile(0.75) + 1.5*IQR
imputer = EndTailImputer(distribution='skewed', tail='right',
                         variables=[columns])

## for leftward skewed dataset --> df.quantile(0.25) - 1.5*IQR
imputer = EndTailImputer(distribution='skewed', tail='left',
                         variables=[columns])

imputer.fit_transform(df)
or 
df.dropna(df.mean()+3*df.std()) or df.dropna(3*df.max())
imputer.imputer_dict_ --> check value 

#### 8. Random Sampling Imputer : 
- Preserves the original distribution

In [ ]:
from feature_engine.missing_data_imputers import RandomSampleImputer
imputer = RandomSampleImputer()
imputer.fit(df)


#### 9. Replacing with Binary values (1/0) : 

In [ ]:
--> np.where(df.isnull(),1,0) # using pandas 
## feature engine function 
from sklearn.impute import MissingIndicator
from feature_engine.missing_data_imputers import AddNaNBinaryImputer
imputer = AddNaNBinaryImputer()
imputer.fit(df)

### 10. Performing multivariate imputation by chained equations(MICE) 
- Each variable with missing data can be modeled based on the remaining variable by using multiple approaches, for example, linear regression, Bayes, decision trees, k-nearest neighbors, and random forests
- Sckit-learn have iterativeImputer function.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import BayesianRidge
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

## 10 iterations are performed 
imputer = IterativeImputer(estimator = BayesianRidge(), max_iter=10, random_state=0)
imputer.fit(df)
imputer.transform(df)

## We can use any algorithm inside iterative imputer 
imputer = IterativeImputer(estimator = LinearRegression(), max_iter=10, random_state=0)
## DecisionTree , KNN , Random Forest 

###  Assembling an imputation pipeline with scikit-learn


In [5]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split


In [4]:
path = r"C:\Users\manev\Desktop\iNeuron\dataset\missing_values\numer_data.csv"
df = pd.read_csv(path)
df.head()

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1.0,f,g,202.0,0.0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6.0,f,g,43.0,560.0,1
2,a,24.50,0.500,NaN,g,q,h,1.50,NaN,NaN,NaN,f,g,280.0,824.0,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5.0,t,g,100.0,3.0,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0.0,f,s,120.0,0.0,1


In [8]:
X_train, X_test, y_train, y_test = train_test_split( df.drop('A15', axis=1),
                                df['A15'], test_size=0.3,random_state=0)


In [9]:
## Creating preferences for Imputation : 
features_num_arbitrary = ['A2', 'A7']
features_num_median = ['A1', 'A13']
features_cat_frequent = ['A3', 'A4', 'A5', 'A6']
features_cat_missing = ['A0', 'A8', 'A9']


In [12]:
## A2 and A7 
imputer_num_arbitrary = Pipeline(steps=[
 ('imputer', SimpleImputer(strategy='constant', fill_value=99)),
])
# A1 and A13 
imputer_num_median = Pipeline(steps=[
 ('imputer', SimpleImputer(strategy='median')),
])
# A3 , A4 , A5 , A6 
imputer_cat_frequent = Pipeline(steps=[
 ('imputer', SimpleImputer(strategy='most_frequent')),
])
# 'A0', 'A8', 'A9'
imputer_cat_missing = Pipeline(steps=[
 ('imputer', SimpleImputer(strategy='constant',
fill_value='Missing')),
])

In [13]:
## Applying Column Transformer : 
preprocessor = ColumnTransformer(transformers=[
 ('imp_num_arbitrary', imputer_num_arbitrary,features_num_arbitrary),
    
 ('imp_num_median', imputer_num_median, features_num_median),
    
 ('imp_cat_frequent', imputer_cat_frequent,features_cat_frequent),
    
 ('imp_cat_missing', imputer_cat_missing, features_cat_missing),
    
 ], remainder='passthrough')


In [18]:
preprocessor.fit_transform(X_train)

array([[3.0, 2.375, 46.08, ..., 't', 'g', 4159.0],
       [2.875, 0.085, 15.92, ..., 'f', 'g', 0.0],
       [2.125, 0.085, 36.33, ..., 'f', 'g', 1187.0],
       ...,
       [0.665, 1.665, 19.58, ..., 'f', 'g', 5.0],
       [2.29, 2.29, 22.83, ..., 't', 'g', 2384.0],
       [3.29, 3.5, 40.58, ..., 't', 's', 0.0]], dtype=object)

In [21]:
X_train_new = pd.DataFrame(preprocessor.fit_transform(X_train),columns=X_train.columns)
X_train_new.head()

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14
0,3.0,2.375,46.08,396.0,u,g,c,v,a,t,t,8.0,t,g,4159.0
1,2.875,0.085,15.92,120.0,u,g,q,v,a,f,f,0.0,f,g,0.0
2,2.125,0.085,36.33,50.0,y,p,w,v,b,t,t,1.0,f,g,1187.0
3,0.585,0.0,22.17,100.0,y,p,ff,ff,b,f,f,0.0,f,g,0.0
4,7.04,14.0,57.83,360.0,u,g,m,v,b,t,t,6.0,t,g,1332.0
